
# Tune side model

In [4]:
import lightgbm as lgb
import optuna
import pandas as pd
import numpy as np
from jesse import helpers, research
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelBinarizer

from custom_indicators.all_features import feature_bundle
from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

_, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
_, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
_, trading_1h = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1h",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

features_3m = feature_bundle(trading_3m, sequential=True)
features_3m = pd.DataFrame(
    {f"3m_{k}": v for k, v in features_3m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_3m[:, 0]]),
)
features_3m = features_3m.resample("15min").agg(
    {k: "last" for k in features_3m.columns}
)
features_15m = feature_bundle(trading_15m, sequential=True)
features_15m = pd.DataFrame(
    {f"15m_{k}": v for k, v in features_15m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_15m[:, 0]]),
)
features_1h = feature_bundle(trading_1h, sequential=True)
features_1h = pd.DataFrame(
    {f"1h_{k}": v for k, v in features_1h.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_1h[:, 0]]),
)
features_1h = (
    features_1h.resample("15min").agg({k: "last" for k in features_1h.columns}).ffill()
)

df_features = pd.concat([features_3m, features_15m, features_1h], axis=1)
print(df_features.shape)


def get_side_label(candles, num_hours, target_ret, pt_sl):
    labeler = TripleBarrierLabeler(
        candles, num_hours=num_hours, num_minutes=1, verbose=False
    )
    side_labels = labeler.side_labels(pt=pt_sl, sl=pt_sl, target_ret=target_ret)
    candle_df = expand_labels(side_labels, candles, fill=0)
    return candle_df

(163680, 1263)


In [5]:
df_features.to_parquet("data/features.parquet")

In [7]:
def objective(trial):
    label_encoder = LabelBinarizer()
    num_hours = trial.suggest_int("num_hours", 1, 24)
    target_ret = trial.suggest_float("target_ret", 0.0005, 0.05)
    pt_sl = trial.suggest_float("pt_sl", 0.1, 3)
    label_df = get_side_label(trading_15m, num_hours, target_ret, pt_sl)
    label_df = label_df.iloc[240:]
    side_features = df_features.iloc[240:]
    valid_mask = label_df["ret"].notna()
    label_df = label_df[valid_mask]
    side_features = side_features[valid_mask]

    train_test_split_point = int(len(label_df) * 0.75)
    train_features = side_features.iloc[:train_test_split_point]
    test_features = side_features.iloc[train_test_split_point:]
    train_labels = label_df.iloc[:train_test_split_point]
    test_labels = label_df.iloc[train_test_split_point:]

    train_labels = train_labels["bin"].astype(int)
    test_labels = test_labels["bin"].astype(int)
    if test_labels.nunique() <= 1:
        return 0

    params = {
        "objective": "multiclassova",
        "num_class": 3,
        "num_threads": -1,
        "verbose": -1,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 200),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
    }
    dtrain = lgb.Dataset(train_features, train_labels)
    # dtest = lgb.Dataset(test_features, test_labels)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 300, 800),
    )
    label_encoder.fit(train_labels)
    pred_proba = model.predict(test_features)
    pred_labels = label_encoder.transform(np.argmax(pred_proba, axis=1) - 1)
    f1_array = f1_score(label_encoder.transform(test_labels), pred_labels, average=None)
    return np.mean(f1_array)


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=500, show_progress_bar=True, n_jobs=1)

/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_65543/3546330371.py:49: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-21 09:24:42,459] A new study created in memory with name: no-name-4db7a6a2-8ca9-4d3e-8a65-cff812d33f44
Best trial: 0. Best value: 0.127451:   0%|          | 1/500 [03:08<26:03:55, 188.05s/it]

[I 2025-02-21 09:27:50,505] Trial 0 finished with value: 0.12745145662355906 and parameters: {'num_hours': 6, 'target_ret': 0.0033746083800556895, 'pt_sl': 0.8519152189168162, 'num_leaves': 141, 'max_depth': 44, 'num_boost_round': 615}. Best is trial 0 with value: 0.12745145662355906.


Best trial: 0. Best value: 0.127451:   0%|          | 2/500 [05:09<20:37:10, 149.06s/it]

[I 2025-02-21 09:29:52,271] Trial 1 finished with value: 0.02842008939569915 and parameters: {'num_hours': 6, 'target_ret': 0.0031138935473136705, 'pt_sl': 2.1729922109138817, 'num_leaves': 121, 'max_depth': 37, 'num_boost_round': 365}. Best is trial 0 with value: 0.12745145662355906.


Best trial: 0. Best value: 0.127451:   1%|          | 3/500 [07:56<21:42:12, 157.21s/it]

...

[I 2025-02-21 13:38:17,059] Trial 497 finished with value: 0.02306168807852653 and parameters: {'num_hours': 23, 'target_ret': 0.04500706761950133, 'pt_sl': 0.3089596926218774, 'num_leaves': 129, 'max_depth': 54, 'num_boost_round': 557}. Best is trial 360 with value: 0.34782608695652173.


Best trial: 360. Best value: 0.347826: 100%|█████████▉| 499/500 [4:13:50<00:18, 18.37s/it]

[I 2025-02-21 13:38:32,585] Trial 498 finished with value: 0.3450740173557938 and parameters: {'num_hours': 23, 'target_ret': 0.04657558880796994, 'pt_sl': 0.1752869101262778, 'num_leaves': 129, 'max_depth': 52, 'num_boost_round': 620}. Best is trial 360 with value: 0.34782608695652173.


Best trial: 360. Best value: 0.347826: 100%|██████████| 500/500 [4:14:09<00:00, 30.50s/it]

[I 2025-02-21 13:38:51,768] Trial 499 finished with value: 0.010172691432061747 and parameters: {'num_hours': 22, 'target_ret': 0.046478543735207725, 'pt_sl': 0.23330965931754977, 'num_leaves': 128, 'max_depth': 52, 'num_boost_round': 612}. Best is trial 360 with value: 0.34782608695652173.


In [ ]:
from jesse import helpers, research

from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

warmup_3m, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

num_minutes = 10
pt_sl = 0.9656765406603607
target_ret = 0.005978786042921383
labeler = TripleBarrierLabeler(trading_3m, num_minutes=10, verbose=True)
side_labels = labeler.side_labels(pt=pt_sl, sl=pt_sl, target_ret=target_ret)
print(side_labels["bin"].value_counts().sort_index().to_numpy())
candle_df = expand_labels(side_labels, trading_3m, fill=0)

In [6]:
candle_df.to_parquet("data/label_side.parquet")

# Tune meta model

In [7]:
import lightgbm as lgb
import optuna
import pandas as pd
from jesse import helpers, research
from sklearn.metrics import roc_auc_score

from custom_indicators.all_features import feature_bundle
from custom_indicators.selection import SIDE_1M, SIDE_3M, SIDE_15M
from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_3m, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_15m, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

features_1m = feature_bundle(trading_1m, sequential=True)
features_1m = pd.DataFrame(
    {f"1m_{k}": v for k, v in features_1m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_1m[:, 0]]),
)
features_1m = features_1m.resample("3T").agg({k: "last" for k in features_1m.columns})
features_3m = feature_bundle(trading_3m, sequential=True)
features_3m = pd.DataFrame(
    {f"3m_{k}": v for k, v in features_3m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_3m[:, 0]]),
)
features_15m = feature_bundle(trading_15m, sequential=True)
features_15m = pd.DataFrame(
    {f"15m_{k}": v for k, v in features_15m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_15m[:, 0]]),
)
features_15m = (
    features_15m.resample("3T").agg({k: "last" for k in features_15m.columns}).ffill()
)

df_features = pd.concat([features_1m, features_3m, features_15m], axis=1)
print(df_features.shape)

side_model = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_features_col = SIDE_1M + SIDE_3M + SIDE_15M
side_model_pred = side_model.predict(df_features[side_features_col])
df_features["model_side_res"] = side_model_pred


def get_meta_label(candles, target_ret, pt, sl, num_minutes=0, num_hours=0):
    labeler = TripleBarrierLabeler(
        candles, num_minutes=num_minutes, num_hours=num_hours, verbose=False
    )
    side_labels = labeler.side_labels(pt=pt, sl=sl, target_ret=target_ret)
    side_full_df = expand_labels(side_labels, candles)
    side_full_df["pred"] = [1 if i > 0.5 else -1 for i in side_model_pred]
    meta_labels = labeler.meta_labels(side_full_df, pt=pt, sl=sl, target_ret=target_ret)
    candle_df = expand_labels(meta_labels, candles, fill=0)
    return candle_df

(818400, 1263)


In [ ]:
def objective(trial):
    num_minutes = trial.suggest_int("num_minutes", 10, 120)
    target_ret = trial.suggest_float("target_ret", 0.0005, 0.01)
    pt = trial.suggest_float("pt", 0.1, 1.2)
    sl = trial.suggest_float("sl", 0.1, 1.2)
    label_df = get_meta_label(trading_3m, num_minutes, target_ret, pt, sl)

    train_test_split_point = int(len(label_df) * 0.8)
    train_features = df_features.iloc[:train_test_split_point]
    test_features = df_features.iloc[train_test_split_point:]
    train_labels = label_df.iloc[:train_test_split_point]
    test_labels = label_df.iloc[train_test_split_point:]

    train_valid_mask = train_labels["ret"].notna().tolist()
    test_valid_mask = test_labels["ret"].notna().tolist()

    train_features = train_features[train_valid_mask]
    train_labels = train_labels[train_valid_mask]["bin"].astype(int)
    test_features = test_features[test_valid_mask]
    test_labels = test_labels[test_valid_mask]["bin"].astype(int)
    if test_labels.nunique() <= 1:
        return 0

    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 100,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 200),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1),
    }
    dtrain = lgb.Dataset(train_features, train_labels)
    dtest = lgb.Dataset(test_features, test_labels)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 300, 800),
    )
    pred_proba = model.predict(test_features)
    auc = roc_auc_score(test_labels, pred_proba, average="weighted")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=500, show_progress_bar=True, n_jobs=1)

In [2]:
{
    "num_minutes": 12,
    "target_ret": 0.00999169655418248,
    "pt": 1.0586409809460318,
    "sl": 0.6472137540480188,
    "num_leaves": 43,
    "max_depth": 71,
    "min_gain_to_split": 0.5374875726311118,
    "min_data_in_leaf": 46,
    "lambda_l1": 0.8426077111660407,
    "lambda_l2": 0.6195771592283434,
    "num_boost_round": 707,
}

{'num_minutes': 12,
 'target_ret': 0.00999169655418248,
 'pt': 1.0586409809460318,
 'sl': 0.6472137540480188,
 'num_leaves': 43,
 'max_depth': 71,
 'min_gain_to_split': 0.5374875726311118,
 'min_data_in_leaf': 46,
 'lambda_l1': 0.8426077111660407,
 'lambda_l2': 0.6195771592283434,
 'num_boost_round': 707}

In [8]:
df_meta_label = get_meta_label(
    trading_3m, 12, 0.00999169655418248, 1.0586409809460318, 0.6472137540480188
)
print(df_meta_label.shape)
df_meta_label.head()

(818400, 5)


,close,ret,trgt,bin,side
2020-06-01 00:00:00+00:00,9439.99,NaN,NaN,0.0,0.0
2020-06-01 00:03:00+00:00,9436.93,NaN,NaN,0.0,0.0
2020-06-01 00:06:00+00:00,9465.00,NaN,NaN,0.0,0.0
2020-06-01 00:09:00+00:00,9475.51,NaN,NaN,0.0,0.0
2020-06-01 00:12:00+00:00,9481.59,NaN,NaN,0.0,0.0


In [10]:
df_meta_label.to_parquet("data/label_meta.parquet")